In [1]:
import glob
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np


In [2]:
def apply_moving_average_threshold(image, window_size):
    blurred_image = cv2.blur(image, (window_size, window_size))
    adaptive_thresholded_image = cv2.adaptiveThreshold(
        image,
        255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        blockSize=window_size,
        C=0  # You can adjust this value based on your image characteristics
    )
    return adaptive_thresholded_image


In [3]:
def parse_dataset():
    train_points = []
    train_labels = []
    class_map = {}
    folder_path = "D:/Amrita/3rdYr/Digital Image Processing/archive/TrainData"
    folders = glob.glob(folder_path + "/*")

    for i, folder in enumerate(folders):
        print("processing class: {}".format(os.path.basename(folder)))
        # store folder name with ID so we can retrieve later
        class_map[i] = os.path.basename(folder)
        # gather all files
        train_files = glob.glob(os.path.join(folder_path, folder) + "/*")
        count = 0

        for f in train_files:
            gray_img = cv2.imread(f,cv2.IMREAD_GRAYSCALE)

            # Apply thresholding
            #_, threshold = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            threshold = apply_moving_average_threshold(gray_img, 65)
            # Obtain Region of Interest (ROI)
            #plt.imshow(threshold,cmap="gray")
            roi = cv2.bitwise_and(gray_img, gray_img, mask=threshold)
            #plt.imshow(roi,cmap="gray")
            # Apply Gaussian Blur
            roi = cv2.GaussianBlur(roi, (9, 9), 0)

            # Apply histogram equalization
            roi = cv2.equalizeHist(roi)
            #plt.imshow(roi,cmap="gray")

            # Remove background noise (you may need to implement this function)
            # roi = removeBackgroundNoise(roi)
            #plt.imshow(roi,cmap="gray")
            # Merge the original image with the preprocessed ROI
            model_input = cv2.merge([gray_img, roi])

            # Invert pixel values
            model_input = 255 - model_input

            # Resize to 128x128
            model_input = cv2.resize(model_input, (128, 128))

            train_points.append(model_input)
            train_labels.append(i)
            
            count+=1
            if(count>60):
                break

    return (
        np.array(train_points),
        np.array(train_labels),
        class_map,
    )
     

In [4]:
train_X, train_y, class_map = parse_dataset()

processing class: BacterialPneumonia
processing class: COVID-19
processing class: Normal
processing class: ViralPneumonia


In [5]:
train_X.shape

(243, 128, 128, 2)

In [8]:
train_y.shape

(243,)

In [9]:
train_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3])

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model
model = Sequential()

# Layer 1
model.add(Conv2D(24, kernel_size=(3, 3), activation='relu', input_shape=(128,128, 2)))
model.add(Dropout(0.2))

# Layer 2
model.add(Conv2D(36, kernel_size=(3, 3), activation='relu'))
model.add(Dropout(0.2))

# Layer 3
model.add(Conv2D(48, kernel_size=(3, 3), activation='relu'))
model.add(Dropout(0.1))

# Layer 5
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Dropout(0.1))

# Flatten layer
model.add(Flatten())

# Dense layers
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 126, 126, 24)      456       
                                                                 
 dropout_5 (Dropout)         (None, 126, 126, 24)      0         
                                                                 
 conv2d_5 (Conv2D)           (None, 124, 124, 36)      7812      
                                                                 
 dropout_6 (Dropout)         (None, 124, 124, 36)      0         
                                                                 
 conv2d_6 (Conv2D)           (None, 122, 122, 48)      15600     
                                                                 
 dropout_7 (Dropout)         (None, 122, 122, 48)      0         
                                                                 
 conv2d_7 (Conv2D)           (None, 120, 120, 64)     

In [11]:
model.fit(train_X,train_y,batch_size=32)

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/sequential_1/dropout_6/dropout/Mul' defined at (most recent call last):
    File "C:\myprograms\python3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\myprograms\python3\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\myprograms\python3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\myprograms\python3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\myprograms\python3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\myprograms\python3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\myprograms\python3\lib\asyncio\base_events.py", line 600, in run_forever
      self._run_on